In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_core.messages import HumanMessage
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from rag import create_vector_search_tool
from langgraph.prebuilt import create_react_agent

In [30]:
settings = {
        "model": "gpt-4o-mini",
        "temperature": 0.5,
        "max_tokens": 2000,
        "frequency_penalty": 0,
        "presence_penalty": 0,
    }

model = ChatOpenAI(
        model=settings["model"],
        temperature=settings["temperature"],
        # max_tokens=settings["max_tokens"] # ChatOpenAI might not take max_tokens directly here
    )
    
def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

loader = PyMuPDFLoader("data/PracticalAdviceOnMatrixGames.pdf")
docs = loader.load()

split_chunks = text_splitter.split_documents(docs)
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
# Create a dummy collection. You'll need to populate this with actual documents for RAG to work.
vector_store = QdrantVectorStore.from_documents(
    split_chunks,
    embedding_function,
    location=":memory:",
    collection_name="matrix_game_docs",
)
# You might want to add some documents here if you have any, e.g.:
# vector_store.add_texts(["Some initial context for the agent"])

# Create the ReAct agent graph
# The search_kwargs for the vector store can be customized if needed
agent_graph = create_react_agent(
        model=model,
        tools=[create_vector_search_tool(vector_store, {"k": 5})]
    )

In [31]:
from typing import Callable, List
from langchain_core.vectorstores import VectorStore
from langchain_core.tools import tool
from langchain_core.documents import Document

from typing import TypedDict

class PoemCollection(TypedDict):
    poem_one: str
    poem_two: str

model_poems = ChatOpenAI(
        model='gpt-4o-mini',
    ).with_structured_output(PoemCollection)


@tool("poem-generator")
async def poem_generator(topic: str) -> str:
    """Generates two poems on the given topic."""
    return await model_poems.ainvoke([HumanMessage(content="Write two poems about " + topic)])


def create_vector_search_tool_test(vector_store: VectorStore, search_kwargs: dict) -> Callable:
  @tool("vector-search", response_format="content_and_artifact")
  def vector_search_tool(query: str) -> List[Document]:
    """Searches a vector database for the given query and returns relevant document contents."""
    retriever = vector_store.as_retriever(search_kwargs=search_kwargs)
    retrieved_docs = retriever.invoke(query)
    return [doc.page_content for doc in retrieved_docs], [doc for doc in retrieved_docs]
  return vector_search_tool

agent_graph = create_react_agent(
        model=model,
        tools=[create_vector_search_tool_test(vector_store, {"k": 5}), poem_generator]
    )

In [24]:
response = model.invoke([HumanMessage(content="Write a poem about cats")])
print(response)

content='In sunlit corners, shadows creep,  \nWhere whispered purrs and dreams do sleep,  \nWith velvet paws and knowing gaze,  \nA world of wonder, they amaze.  \n\nThey stalk like phantoms, soft and sly,  \nWith emerald eyes that catch the sky,  \nIn graceful leaps, they claim their throne,  \nA monarch in a realm of stone.  \n\nEach flick of tail, each playful swipe,  \nA dance of joy, a thrill, a type,  \nOf fleeting moments, fun and free,  \nIn every nook, a mystery.  \n\nOn windowsills, they watch the day,  \nAs leaves and chirps and children play,  \nWith gentle grace, they curl and pounce,  \nA symphony of furry flounce.  \n\nThe night awakens, stars ignite,  \nThey prowl like shadows, sleek and light,  \nWith whispers soft, they roam the dark,  \nA silent echo, a glowing spark.  \n\nIn quiet solitude, they lay,  \nA heartbeat close, a soft array,  \nWith every purr, they weave a spell,  \nIn cozy warmth, all’s right, all’s well.  \n\nSo here’s to cats, with hearts so bold,  \n

/tmp/ipykernel_31368/2504727401.py:1: RuntimeWarning: coroutine 'BaseChatModel.ainvoke' was never awaited
  response = model.invoke([HumanMessage(content="Write a poem about cats")])


In [ ]:
response = agent_graph.invoke({"messages": [HumanMessage(content="How are random events handled in matrix games?")]})
print(response)

In [34]:
async for chunk in agent_graph.astream(
    {"messages": [{"role": "user", "content": "Write a poem about cats. Use your poem generator tool."}]}
):
    print(chunk)
    print("\n")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5hlgkS8h0yL9Ig7wVUpjPNjI', 'function': {'arguments': '{"topic":"cats"}', 'name': 'poem-generator'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 85, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BW5Lb5h3Tie17QRZ8J1dGvx37XiP7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--66612da8-ac29-4b9f-8345-16f0683f5919-0', tool_calls=[{'name': 'poem-generator', 'args': {'topic': 'cats'}, 'id': 'call_5hlgkS8h0yL9Ig7wVUpjPNjI', 'type': 'tool_call'}], usage_metadata={'input_tokens': 85, 'output_tokens': 16, 'total_tokens': 101,

In [32]:
async for stream_mode, chunk in agent_graph.astream(
    {"messages": [{"role": "user", "content": "Write a poem about cats. Use your poem generator tool."}]},
    stream_mode=["updates", "messages"]
):
    print(chunk)
    print("\n")

(AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_gAGabVgJlSFjE894FEuey9xc', 'function': {'arguments': '', 'name': 'poem-generator'}, 'type': 'function'}]}, response_metadata={}, id='run--5b5ff2a5-6018-47de-bfb4-bd34607b48c8', tool_calls=[{'name': 'poem-generator', 'args': {}, 'id': 'call_gAGabVgJlSFjE894FEuey9xc', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'poem-generator', 'args': '', 'id': 'call_gAGabVgJlSFjE894FEuey9xc', 'index': 0, 'type': 'tool_call_chunk'}]), {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:51697eab-30f9-ac6a-0e7c-c03c1b438598', 'checkpoint_ns': 'agent:51697eab-30f9-ac6a-0e7c-c03c1b438598', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.5})


(AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'a

In [15]:
async for stream_mode, chunk in agent_graph.astream(
    {"messages": [{"role": "user", "content": "How are random events handled in matrix games?"}]},
    stream_mode=["updates", "messages"]
):
    print(chunk)
    print("\n")

(AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_xNEJWAEc0MbVXCikNMGECQ4R', 'function': {'arguments': '', 'name': 'vector-search'}, 'type': 'function'}]}, response_metadata={}, id='run--4c8f5503-9985-4fc9-8ee9-2f219e894486', tool_calls=[{'name': 'vector-search', 'args': {}, 'id': 'call_xNEJWAEc0MbVXCikNMGECQ4R', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'vector-search', 'args': '', 'id': 'call_xNEJWAEc0MbVXCikNMGECQ4R', 'index': 0, 'type': 'tool_call_chunk'}]), {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:a52933d6-7d84-fecc-25c0-0cf0562c199a', 'checkpoint_ns': 'agent:a52933d6-7d84-fecc-25c0-0cf0562c199a', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.5})


(AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'argu

In [9]:
async for token, metadata in agent_graph.astream(
    {"messages": [{"role": "user", "content": "How are random events handled in matrix games?"}]},
    stream_mode="messages"
):
    print("token: ", token)
    print("metadata: ", metadata)
    print("\n")

token:  content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8XWuvrWzzG0pF8YJcg58Q54d', 'function': {'arguments': '', 'name': 'vector-search'}, 'type': 'function'}]} response_metadata={} id='run--59d4fdce-2a72-40ae-b79b-00cd7b53cbe4' tool_calls=[{'name': 'vector-search', 'args': {}, 'id': 'call_8XWuvrWzzG0pF8YJcg58Q54d', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'vector-search', 'args': '', 'id': 'call_8XWuvrWzzG0pF8YJcg58Q54d', 'index': 0, 'type': 'tool_call_chunk'}]
metadata:  {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:26efaa58-49df-e49e-fce7-dd8fd1c3d623', 'checkpoint_ns': 'agent:26efaa58-49df-e49e-fce7-dd8fd1c3d623', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.5}


token:  content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 

In [18]:
async for token, metadata in agent_graph.astream(
    {"messages": [{"role": "user", "content": "How are random events handled in matrix games?"}]},
    stream_mode="messages"
):
    print(token.content)
    print("metadata: ", metadata)


metadata:  {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.5}

metadata:  {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.5}

metadata:  {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f809900

In [12]:
async for token, metadata in agent_graph.astream(
            {"messages": [HumanMessage(content="How are random events handled in matrix games?")]}, stream_mode = "messages"
        ):
            await print(token)
            await print(metadata)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_qah9XeCDgY6byTkPLwfBlanw', 'function': {'arguments': '', 'name': 'vector-search'}, 'type': 'function'}]} response_metadata={} id='run--d6a9582a-569c-415a-83c4-6c2bb868f99f' tool_calls=[{'name': 'vector-search', 'args': {}, 'id': 'call_qah9XeCDgY6byTkPLwfBlanw', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'vector-search', 'args': '', 'id': 'call_qah9XeCDgY6byTkPLwfBlanw', 'index': 0, 'type': 'tool_call_chunk'}]


TypeError: object NoneType can't be used in 'await' expression

In [4]:
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper(doc_content_chars_max = 400000)

In [9]:
wikipedia.load("India Pakistan conflict 2025")

[Document(metadata={'title': '2025 India–Pakistan standoff', 'summary': "On 23 April 2025, a standoff emerged between India and Pakistan, which was sparked by the 2025 Pahalgam attack, a terrorist attack in the Baisaran Valley of Jammu and Kashmir, killing 27 people, including 25 Hindu tourists, a Christian tourist, and a local Muslim, as well as injuring more than 20 others. The Resistance Front (TRF) initially claimed responsibility for the attack. Armed skirmishes between India and Pakistan were reported along the Line of Control (LoC) beginning on 24 April, raising fears of further escalation between the two nuclear-armed neighbours. A ceasefire was announced on 10 May 2025 from 16:30 PKT/17:00 IST following an agreement between India and Pakistan.\nThe standoffs were followed by a diplomatic crisis which emerged between the two countries, as India accused Pakistan of sponsoring the attack. India initiated the expulsion of Pakistani diplomats, recalled its own diplomatic staff, sus